In [1]:
import os

with open("/Users/miriambenali/Desktop/Project-Simplon/AI-Assistant-using-RAG-LangChain-for-support-IT/ml/data_knowledge/it_support_questions.txt", "r",encoding="utf-8") as f:
    
    questions_it= [line.strip() for line in f if line.strip()]

    print(len(questions_it))      
    print(type(questions_it)) 

101
<class 'list'>


In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embedding= model.encode(questions_it)

print(embedding.shape)

/Users/miriambenali/Desktop/Project-Simplon/AI-Assistant-using-RAG-LangChain-for-support-IT/myvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/miriambenali/Desktop/Project-Simplon/AI-Assistant-using-RAG-LangChain-for-support-IT/myvenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/miriambenali/Desktop/Project-Simplon/AI-Assistant-using-RAG-LangChain-for-support-IT/myvenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you

(101, 384)


In [3]:
from sklearn.cluster import KMeans

kmeans= KMeans(n_clusters=3, random_state=42)
labels= kmeans.fit_predict(embedding)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/miriambenali/Desktop/Project-Simplon/AI-Assistant-using-RAG-LangChain-for-support-IT/myvenv/lib/python3.11/site-packages/threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [4]:
for q, label in zip(questions_it[:10], labels[:10]):
    print(f"[Cluster {label}] {q}")

[Cluster 1] What are the three fundamental questions of IT support
[Cluster 1] Why is the question How considered the most important fundamental question
[Cluster 2] How does the process of elimination work in IT troubleshooting
[Cluster 2] What does it mean to begin at the end when diagnosing a technical problem
[Cluster 1] Why is it important to ask what has changed at the start of a support ticket
[Cluster 2] How do you identify the root cause of an intermittent network issue
[Cluster 2] What are the benefits of using a holistic approach to troubleshooting
[Cluster 2] How can a technician avoid overtaxing themselves during complex repairs
[Cluster 0] What is the significance of the dots in a hardware point chain
[Cluster 2] How should a technician handle a problem that seems impossible to solve


In [5]:
#Create Categories

from collections import defaultdict

cluster_names = {
    0: "Hardware & Physical Connections",
    2: "Diagnostic & Root Cause Analysis",
    3: "Ticket Context & Change Management",
    4: "Troubleshooting Methodology"
}

clusters= defaultdict(list)

for question, label in zip(questions_it,labels):
    clusters[label].append(question)

for label, qs in clusters.items():
    print(f"\n Cluster {label} – {cluster_names.get(label, 'Unknown')}")
   


 Cluster 1 – Unknown

 Cluster 2 – Diagnostic & Root Cause Analysis

 Cluster 0 – Hardware & Physical Connections


In [6]:
import joblib

joblib.dump(kmeans, "kmeans_it_support.pkl")

['kmeans_it_support.pkl']

In [7]:
import chromadb

#client = chromadb.PersistentClient(path="./ml/data_knowledge/chroma_db_question")